This notebook contains all the code needed to generate a synthetic dataset.

Exits and funding are pulled from lognormal distr
Ordinal founder attrs come from discrete pdfs in hyperparameter config.

All params handled in `src/config.py` `SYNTH` variable.

Visualizations provided at the end to compare with expectations of real-world data.

Defining success as an exit or >5m in funding. compute a raw score (which could be used to prioritize) and probability of success in `model.ipynb`.

In [41]:
# ! pip install seaborn plotly scipy -q

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
import sys

sys.path.append("..")

from src.config.config import cfg
from src.datagen import datagen

In [ ]:
MATRIX = cfg.MATRIX
POPULATIONS = cfg.SYNTH["POPULATIONS"]

K = sum(cfg["DIMENSION"] for cfg in MATRIX.values())

dg = datagen.DataGenerator()

In [45]:
# viz tools
def get_category_slice(category_name):
    start = 0
    for cat in MATRIX:
        d = MATRIX[cat]["DIMENSION"]
        if cat == category_name:
            return slice(start, start + d)
        start += d
    return slice(0, 0)  # fallback


def plot_category_distribution_overall_and_by_pop(X, pop_labels, category_name):
    cat_slice = get_category_slice(category_name)
    d = MATRIX[category_name]["DIMENSION"]

    cat_indices = X[:, cat_slice].argmax(axis=1)  # 0..(d-1)

    counts = np.bincount(cat_indices, minlength=d)

    plt.figure(figsize=(8, 5))
    bars = plt.bar(range(d), counts, color="cornflowerblue", edgecolor="black")
    plt.title(f"Overall Distribution of {category_name} (tiers=0..{d-1})", fontsize=14)
    plt.xlabel("Tier Index", fontsize=12)
    plt.ylabel("Count", fontsize=12)

    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.5,
            f"{int(height)}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

    plt.tight_layout()
    plt.show()

    unique_pops = np.unique(pop_labels)
    plt.figure(figsize=(10, 6))
    width = 0.8 / len(unique_pops)

    pop_percentages = {}
    for pop_name in unique_pops:
        pop_mask = pop_labels == pop_name
        cat_pop_indices = cat_indices[pop_mask]
        ccounts = np.bincount(cat_pop_indices, minlength=d)
        pop_percentages[pop_name] = 100.0 * ccounts / ccounts.sum()

    for i, pop_name in enumerate(unique_pops):
        pop_mask = pop_labels == pop_name
        x_positions = np.arange(d) + (i - len(unique_pops) / 2) * width + width / 2
        bars = plt.bar(
            x_positions,
            pop_percentages[pop_name],
            width=width,
            label=f"{pop_name} (n={pop_mask.sum()})",
        )

        for bar in bars:
            height = bar.get_height()
            if height > 5:
                plt.text(
                    bar.get_x() + bar.get_width() / 2.0,
                    height + 0.5,
                    f"{height:.1f}%",
                    ha="center",
                    va="bottom",
                    fontsize=8,
                )

    plt.title(f"{category_name} Tier Distribution by Population (%)", fontsize=14)
    plt.xlabel("Tier Index", fontsize=12)
    plt.ylabel("Percentage within Population", fontsize=12)
    plt.xticks(np.arange(d), [str(i) for i in range(d)])
    plt.legend(loc="best")
    plt.grid(axis="y", linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.show()


def plot_hist_overall_and_by_pop(values, pop_labels, title, bins=50, log_scale=False):
    bins = int(bins)

    if log_scale:
        raw_vals = values
        plot_vals = np.log1p(values)
        xlabel_str = f"{title} [log(1 + x)]"
    else:
        raw_vals = values
        plot_vals = values
        xlabel_str = title

    plt.figure(figsize=(9, 5))

    min_val, max_val = plot_vals.min(), plot_vals.max()
    bin_edges = np.linspace(min_val, max_val, bins)

    n, bin_edges, patches = plt.hist(
        plot_vals, bins=bin_edges, color="skyblue", edgecolor="black", alpha=0.7
    )
    plt.title(f"{title} - Overall Distribution", fontsize=14)
    plt.xlabel(xlabel_str, fontsize=12)
    plt.ylabel("Count", fontsize=12)

    stats_text = (
        f"Mean: {raw_vals.mean():.2f}\n"
        f"Median: {np.median(raw_vals):.2f}\n"
        f"Std: {raw_vals.std():.2f}"
    )
    plt.annotate(
        stats_text,
        xy=(0.70, 0.75),
        xycoords="axes fraction",
        bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.8),
    )

    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

    unique_pops = np.unique(pop_labels)
    plt.figure(figsize=(10, 6))

    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_pops)))

    for i, pop_name in enumerate(unique_pops):
        mask = pop_labels == pop_name
        sub_vals_raw = raw_vals[mask]
        sub_vals_log = plot_vals[mask]

        label_str = (
            f"{pop_name} (n={mask.sum()}, "
            f"mean={sub_vals_raw.mean():.2f}, "
            f"std={sub_vals_raw.std():.2f})"
        )

        plt.hist(
            sub_vals_log, bins=bin_edges, alpha=0.6, label=label_str, color=colors[i]
        )

    plt.xlabel(xlabel_str, fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.title(f"{title} by Population", fontsize=14)
    plt.legend(loc="best")
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))

    for i, pop_name in enumerate(unique_pops):
        mask = pop_labels == pop_name
        sub_vals_raw = raw_vals[mask]
        sub_vals_log = plot_vals[mask]

        if len(sub_vals_log) > 1:
            kde = stats.gaussian_kde(sub_vals_log)
            x_eval = np.linspace(sub_vals_log.min(), sub_vals_log.max(), 500)
            density = kde(x_eval)

            label_str = f"{pop_name} (mean={sub_vals_raw.mean():.2f})"
            plt.plot(x_eval, density, label=label_str, color=colors[i], linewidth=2)

    plt.xlabel(xlabel_str, fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.title(f"{title} Density by Population (KDE)", fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()


def plot_scatter_overall_and_by_pop(
    xvals, yvals, pop_labels, x_title, y_title, log_y=False
):
    plt.figure(figsize=(10, 7))
    unique_pops = np.unique(pop_labels)
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_pops)))

    for i, pop_name in enumerate(unique_pops):
        mask = pop_labels == pop_name
        x_pop = xvals[mask]
        y_pop = yvals[mask]

        plt.scatter(x_pop, y_pop, alpha=0.5, label=pop_name, color=colors[i])

        if len(x_pop) > 3:
            if log_y:
                log_y_pop = np.log(y_pop)
                z = np.polyfit(x_pop, log_y_pop, 1)
                p = np.poly1d(z)
                x_range = np.linspace(x_pop.min(), x_pop.max(), 50)
                plt.plot(
                    x_range,
                    np.exp(p(x_range)),
                    "--",
                    color=colors[i],
                    linewidth=2,
                    alpha=0.7,
                )
            else:
                z = np.polyfit(x_pop, y_pop, 1)
                p = np.poly1d(z)
                x_range = np.linspace(x_pop.min(), x_pop.max(), 50)
                plt.plot(
                    x_range, p(x_range), "--", color=colors[i], linewidth=2, alpha=0.7
                )

    if log_y:
        plt.yscale("log")

    plt.xlim(left=0)

    plt.title(f"{x_title} vs. {y_title} by Population", fontsize=14)
    plt.xlabel(x_title, fontsize=12)
    plt.ylabel(y_title + (" [log scale]" if log_y else ""), fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()


def plot_feature_importance(W_star):
    feature_importance = np.sum(np.abs(W_star), axis=0)

    feature_names = []
    for cat_name in MATRIX:
        dim = MATRIX[cat_name]["DIMENSION"]
        for i in range(dim):
            feature_names.append(f"{cat_name}_{i}")

    sorted_indices = np.argsort(feature_importance)[::-1]
    sorted_imp = feature_importance[sorted_indices]
    sorted_names = [feature_names[i] for i in sorted_indices]

    top_k = min(15, len(sorted_names))
    plt.figure(figsize=(10, 6))
    bars = plt.bar(range(top_k), sorted_imp[:top_k], color="teal")
    plt.xticks(range(top_k), sorted_names[:top_k], rotation=45, ha="right")
    plt.title("Top Feature Importance (from W*)", fontsize=14)
    plt.xlabel("Feature", fontsize=12)
    plt.ylabel("Importance Score", fontsize=12)

    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.5,
            f"{height:.2f}",
            ha="center",
            va="bottom",
            fontsize=8,
        )

    plt.tight_layout()
    plt.show()


def plot_correlation_heatmap(X, y, exit_vals, fund_vals, feature_names=None):
    if feature_names is None:
        feature_names = []
        for cat_name in MATRIX:
            d = MATRIX[cat_name]["DIMENSION"]
            for i in range(d):
                feature_names.append(f"{cat_name}_{i}")

    data = np.column_stack([X, y, exit_vals, fund_vals])
    col_names = feature_names + ["Score", "Exit", "Funding"]
    df = pd.DataFrame(data, columns=col_names)

    corr = df.corr()

    plt.figure(figsize=(12, 10))
    mask = np.zeros_like(corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    sns.heatmap(
        corr,
        mask=mask,
        cmap=cmap,
        center=0,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.5},
        annot=True,
        fmt=".2f",
    )
    plt.title("Correlation Heatmap: Features & Targets", fontsize=16)
    plt.tight_layout()
    plt.show()

    target_cols = ["Score", "Exit", "Funding"]
    target_corr = corr.loc[target_cols, target_cols]
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        target_corr,
        cmap=cmap,
        center=0,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.5},
        annot=True,
        fmt=".2f",
    )
    plt.title("Correlation among Targets", fontsize=14)
    plt.tight_layout()
    plt.show()


def visualize_successful_cases(exit_vals, fund_vals):
    funding_success_mask = fund_vals > 5_000_000
    exit_success_mask = exit_vals > 0
    both_success_mask = funding_success_mask & exit_success_mask
    unsuccessful_mask = ~(funding_success_mask | exit_success_mask)

    num_funding_success = np.sum(funding_success_mask)
    num_exit_success = np.sum(exit_success_mask)
    num_both_success = np.sum(both_success_mask)
    num_unsuccessful = np.sum(unsuccessful_mask)

    # Print the counts
    print(f"Funding > $5M: {num_funding_success}")
    print(f"Exit > 0: {num_exit_success}")
    print(f"Both Funding > $5M and Exit > 0: {num_both_success}")
    print(f"Unsuccessful: {num_unsuccessful}")

    # Plot
    categories = ["Funding > $5M", "Exit > 0", "Both", "Unsuccessful"]
    counts = [num_funding_success, num_exit_success, num_both_success, num_unsuccessful]

    plt.figure(figsize=(8, 5))
    plt.bar(categories, counts, color=["teal", "orange", "purple", "grey"])
    plt.title("Number of Successful vs Unsuccessful Cases", fontsize=14)
    plt.xlabel("Category", fontsize=12)
    plt.ylabel("Number of Cases", fontsize=12)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

### Generate dataset
Configure params in `src/config/config.SYNTH`

In [46]:
N = 10000
X_syn, exit_syn, fund_syn, pop_labels, W_star = dg.generate_synthetic_dataset(N, POPULATIONS)

In [ ]:
# plot
for cat_name in MATRIX.keys():
    plot_category_distribution_overall_and_by_pop(X_syn, pop_labels, cat_name)

plot_hist_overall_and_by_pop(
    fund_syn, pop_labels, title="Funding Amount", bins=50, log_scale=True
)

plot_hist_overall_and_by_pop(
    exit_syn, pop_labels, title="Exit Value", bins=50, log_scale=True
)

plot_feature_importance(W_star)

visualize_successful_cases(exit_syn, fund_syn)

In [ ]:
# Save dataset
feature_names = []
for cat in MATRIX:
    dim = MATRIX[cat]["DIMENSION"]
    for i in range(dim):
        if dim == 3:
            feature_names.append(f"{cat}_{i + 1}")
        else:
            feature_names.append(f"{cat}_{i}")

df = pd.DataFrame(X_syn, columns=feature_names)
df["exit_value"] = exit_syn
df["funding_amount"] = fund_syn

df["success"] = ((df["exit_value"] > 0) | (df["funding_amount"] > 5_000_000)).astype(int)


df.to_csv("../data/synth/encoded_founders_composites.csv", index=False)
df